In [24]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import models 

CAM models provide interpretability to AI, this had important ramifications in for three areas in ai 
-  Identify why a model is not providing the desired result 
-  Understanding the result of the model. Increase customer trust 
-  Training 

The concept behind a CAM-type model logic seems fairly trivial. The outputs of a convolution layer (before applying any activation function) will correspond to the 'weight maps' for the input. The last convolutional layer will output k feature maps, where k is the number of kernels or filters in the layer; the feature map represents the activation of the filter applied to the input image. Note that the input can be anything from an image to a sentence. Changing this map will change the output of the model, hence, that changed can be modeled as the gradient of an output (y) with respect to the weight map (A).

In [2]:
# Toy sample model
model = models.resnet18(pretrained=True)

/Users/aina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/aina/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/aina/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [18]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [25]:
class CAMResNet18(models.resnet.ResNet):
    def __init__(self, *args, **kwargs):
        super(CAMResNet18, self).__init__(block=models.resnet.BasicBlock, layers=[2, 2, 2, 2], *args, **kwargs)
        
        self.classifier2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        # Apply ResNet up to the last convolutional layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        features = self.layer4(x)
        
        x = self.avgpool(features)
        x = x.view(x.size(0), -1)
        output1 = self.fc(x)
        
        output2 = self.classifier2(features.view(features.size(0,1)))
        
        return output1, output2, features


In [27]:
CAMResNet18(num_classes = 2)

NameError: name 'num_classes' is not defined

In [14]:
features1 = model.layer1
features3 = model.layer3
features4 = model.layer4

fc = torch.nn.Linear(512, 10) # create a new fully connected layer 

model.fc = torch.nn.Identity() # remove the last fully connected layer 

x = torch.randn(1, 3, 224, 224) # input
y_pred = model(x)

In [16]:
features4.weight

AttributeError: 'Sequential' object has no attribute 'weight'

In [11]:
features4_map = features4(x)
features1_map = features1(x)

RuntimeError: Given groups=1, weight of size [512, 256, 3, 3], expected input[1, 3, 224, 224] to have 256 channels, but got 3 channels instead

In [ ]:
weights  = fc.weight
cam = torch.matmul(weights, features_map_view(512,-1))